In [2]:
from sklearn.linear_model import Perceptron
import numpy as np
import pickle

priors = np.array([0.13847689,  0.02169942,  0.07486422,  0.01583011,  0.04773386,  0.04875673,
                   0.03044251,  0.13501863,  0.03241519,  0.02121234,  0.01341906,  0.00786635,
                   0.0112272,   0.01339471,  0.00501693,  0.02247875,  0.1585446,   0.06595066,
                   0.0373834,   0.02980931,  0.02856725,  0.02937094,  0.00723314,  0.00328779])

def create_ranking(classifiers, data, priors):

    results = np.hstack([clf.predict(data).reshape(-1, 1) for clf in classifiers])
    results = (results / np.sqrt(priors)).sum(axis=1)
    
    results_pairs = [(r, i) for i, r in enumerate(results)]
    return [pair[1] for pair in reversed(sorted(results_pairs))]



In [3]:
from sklearn.linear_model import Perceptron
import numpy as np
import pickle
print( 'loading data')

with open('train_data.pkl' , 'rb') as fh:
    X = pickle.load(fh )

with open('train_labels.pkl' ,'rb') as fh:
    y = pickle.load(fh , encoding='bytes')

print( 'loaded')
print( ' +', X.shape[0], 'compounds')
print( ' +', y.shape[1], 'proteins')



loading data
loaded
 + 14891 compounds
 + 24 proteins


In [5]:
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
protein= 0
for protein in range(y.shape[1]):
    print( ' + model of protein', (protein+1))
    labels = y[:, protein].reshape(-1)
    labels[labels == -1] = 0
    
    #X_train, X_test, labels_train, labels_test = train_test_split(X, labels, test_size=0.20, random_state=42)
    kf = KFold(len(labels), 5)
    sumR = 0.0
    for train, test in kf:
        X_train, X_test, labels_train, labels_test = X[train], X[test], labels[train], labels[test]
        clf = Perceptron(random_state=0, class_weight='auto', n_iter=100)
        clf.fit(X_train, labels_train)

        labels_predict = clf.predict(X_test)
        
        
        sumR = sumR + sum( labels_predict == labels_test )/float(len(labels_test)) 
    print(sumR/5)

 + model of protein 1
0.622179839129
 + model of protein 2
0.848295805133
 + model of protein 3
0.331282690497
 + model of protein 4
0.947618532323
 + model of protein 5
0.479870533177
 + model of protein 6
0.898529441934
 + model of protein 7
0.595184942459
 + model of protein 8
0.742593746104
 + model of protein 9
0.774359288244
 + model of protein 10
0.763817575953
 + model of protein 11
0.872199554031
 + model of protein 12
0.894763365948
 + model of protein 13
0.930439221855
 + model of protein 14
0.970788084309
 + model of protein 15
0.980256602576
 + model of protein 16
0.798194435145
 + model of protein 17
0.723390259689
 + model of protein 18


KeyboardInterrupt: 

In [19]:
from sklearn.cross_validation import KFold

protein = 0
print( ' + model of protein', (protein+1))
labels = y[:, protein].reshape(-1)
labels[labels == -1] = 0
for i in range(10):
    #X_train, X_test, labels_train, labels_test = train_test_split(X, labels, test_size=0.20, random_state=42)
    kf = KFold(len(labels), 5 , random_state = i*10)
    sumR = 0.0
    for train, test in kf:
        X_train, X_test, labels_train, labels_test = X[train], X[test], labels[train], labels[test]
        clf = Perceptron(random_state=0, class_weight='auto', n_iter=200)
        clf.fit(X_train, labels_train)

        labels_predict = clf.predict(X_test)
        
        
        sumR = sumR + sum( labels_predict == labels_test )/float(len(labels_test)) 
    print(sumR/5)

 + model of protein 1
0.724732270735
0.724732270735
Traceback (most recent call last):
  File "/home/user/anaconda3/lib/python3.4/site-packages/IPython/core/interactiveshell.py", line 3032, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-19-1d77689c8f08>", line 14, in <module>
    clf.fit(X_train, labels_train)
  File "/home/user/anaconda3/lib/python3.4/site-packages/sklearn/linear_model/stochastic_gradient.py", line 483, in fit
    sample_weight=sample_weight)
  File "/home/user/anaconda3/lib/python3.4/site-packages/sklearn/linear_model/stochastic_gradient.py", line 387, in _fit
    classes, sample_weight, coef_init, intercept_init)
  File "/home/user/anaconda3/lib/python3.4/site-packages/sklearn/linear_model/stochastic_gradient.py", line 352, in _partial_fit
    sample_weight=sample_weight, n_iter=n_iter)
  File "/home/user/anaconda3/lib/python3.4/site-packages/sklearn/linear_model/stochastic_gradient.py", line 398, in _fit_binary
    sample_w

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [ ]:
classifiers = []

print 'training classifiers'
for protein in range(y.shape[1]):

    print ' + model of protein', (protein+1)
    labels = y[:, protein].reshape(-1)
    labels[labels == -1] = 0

    clf = Perceptron(random_state=0, class_weight='auto', n_iter=100)
    clf.fit(X, labels)

    classifiers.append(clf)


In [ ]:

print 'predicting'

print ' + training set'
ranking = create_ranking(classifiers, X, priors=priors)
with open('train_output.txt', 'w') as fw:
    for r in ranking:
        fw.write(str(r) + ' ')
    fw.write("\n")


In [ ]:

print ' + validation set'

with open('valid_data.pkl') as fh:
    X = pickle.load(fh)

ranking = create_ranking(classifiers, X, priors=priors)
with open('valid_output.txt', 'w') as fw:
    for r in ranking:
        fw.write(str(r) + ' ')
    fw.write("\n")
# It gets around 63% on validation set

print ' + test set'
with open('test_data.pkl') as fh:
    X = pickle.load(fh)

ranking = create_ranking(classifiers, X, priors=priors)
with open('test_output.txt', 'a') as fw:
    for r in ranking:
        fw.write(str(r) + ' ')
    fw.write("\n")